<a href="https://colab.research.google.com/github/Lucasscheere/analise_de_credito/blob/main/analise2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import timedelta, datetime, date
import plotly.graph_objs as go
import plotly.offline as py
py.init_notebook_mode(connected=False)
import plotly.io as pio
pio.renderers.default = "colab"

In [ ]:
df = pd.read_excel('CLIENTE.xls', usecols=['Data', 'Valor', 'Vencimento','F.P.', 'Pagamento', 'Atraso', 'Valor Pago'])
df.head()

In [ ]:
#Removendo os pedidos da TM e à vista
df = df.loc[(df['F.P.']!="TM")
          & (df['F.P.']!="DI")
          & (df['F.P.']!="CR")
          & (df['F.P.']!="CC")
          & (df['F.P.']!="DB")
          #& (df['F.P.']!="VL")
          ]

In [ ]:
data_final = datetime.today()
data_inicial = df.Data.min()

In [ ]:
array_datas = pd.date_range(start=data_inicial, end=data_final, freq = 'D')

In [ ]:
df2 = pd.DataFrame(array_datas, columns=['Data'])

In [ ]:
def comprasdodia(dia):
    valordacompra = df.loc[df['Data']==dia].Valor.sum()
    return valordacompra

In [ ]:
def pagamentosdodia(dia):
    valordopagamento = df.loc[df['Pagamento']==dia].Valor.sum()
    return valordopagamento

In [ ]:
def vencimentosdodia(dia):
    valordovencimento = df.loc[df['Vencimento']==dia].Valor.sum()
    return valordovencimento

In [ ]:
df2['Compras'] =  df2['Data'].apply(comprasdodia)

In [ ]:
df2['Pagamentos'] =  df2['Data'].apply(pagamentosdodia)

In [ ]:
df2['Vencimentos'] =  df2['Data'].apply(vencimentosdodia)

In [ ]:
df2['compras_acumuladas'] = df2.Compras.cumsum()
df2['vencimentos_acumulados'] = df2.Vencimentos.cumsum()
df2['pagamentos_acumulados'] = df2.Pagamentos.cumsum()

In [ ]:
#Gerando a coluna 'valor_em_aberto', com a diferença entre o que foi comprado e o que foi pago
df2['valor_em_aberto'] = df2['compras_acumuladas'].sub(df2['pagamentos_acumulados'])

#Gerando a coluna 'valor_em_atraso', com a diferença entre o que foi comprado e o que foi pago
df2['valor_em_atraso'] = df2['vencimentos_acumulados'].sub(df2['pagamentos_acumulados'])
df2

In [ ]:
#criando gráfico
linevalues = go.Figure()

linevalues.add_trace(
    go.Scatter(
        x=df2.Data, y=df2.valor_em_aberto,
        mode='lines',
        marker = {'color' : '#0592FF'},
        name='Em aberto'
        )
    )

linevalues.add_trace(
    go.Scatter(
        x=df2.Data,
        y=df2.valor_em_atraso,
        mode='lines',
        marker = {'color' : '#FF494C'},
        name='Em atraso'
        )
    )

#layout do título
linevalues.update_layout(
    title='Valores Em aberto x Em atraso (R$)',
    xaxis={'title': 'Período'},
    yaxis={'title': 'Valores'},
    hovermode="x")

linevalues.update_layout(
    title={
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
        }
    )

#background
linevalues.layout.plot_bgcolor = '#fff'

#grids
linevalues.update_xaxes(
    showgrid=True, gridwidth=1, gridcolor='lightgray',
    showline=True, linewidth=1, linecolor='black'
)

linevalues.update_yaxes(
    showgrid=True, gridwidth=1, gridcolor='lightgray',
    showline=True, linewidth=1, linecolor='black'
)

#display
linevalues.show()

In [ ]:
#Maior valor em aberto:
maiorValorEmAberto = round(df2.valor_em_aberto.max(), 2)

data_maior_valor_em_aberto = df2.loc[df2['valor_em_aberto']==df2.valor_em_aberto.max()].Data.dt.date.values[0].strftime('%d/%m/%Y')

print(f'Maior valor em aberto: R$ {maiorValorEmAberto:,} dia {data_maior_valor_em_aberto}')

In [ ]:
df.Atraso.hist()

In [ ]:
print("Média: ", df.Atraso.mean())
print("Mediana", df.Atraso.median())

In [ ]:
df.boxplot(column=['Atraso'])